## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list


In [ ]:
import importlib
import Python_scripts.Feature_functions.angle_features
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.angle_features)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.angle_features import (
    angle_features_for_trial, batch_angle_features
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


## Fetch id list

In [ ]:
task_name = "ToyOnly"
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=task_name, #['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly'],
    dose_mult=2,
    genotype="white",
    bad_ids=bad_id,
    csv_prefix="dlc_table",   # -> dlc_table_saline.csv, dlc_table_ghrelin.csv, ...
    min_trial_length=None     # or 600
)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"exc_id: {exc_id}\n")
print(f"inh_id: {inh_id}\n")


In [ ]:
ts, sm, vl = angle_features_for_trial(conn, 452)
print(f"[INFO] ts type {type(ts)}")
print(f"[INFO] sm type {type(sm)}")
print(f"[INFO] vl type {type(vl)}")


## Calculate angle features

### Batch call with different input parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
likelihood_thr=np.arange(0.65, 0.68, 0.025)
window_size=None

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    # "Ghrelin": ghrelin_id,
    "Inhibitory": inh_id,
    "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_Modulation_2X_{task_name}_ang_likelihood_sweep.pdf"
with PdfPages(outfile) as pdf:
    for likelihood in likelihood_thr:
        print(f"[INFO] Analyzing likelihood = {likelihood}")

        # Compute all groups for this window size
        frames = []
        for label, ids in group_specs.items():
            df = batch_angle_features(
                conn, ids, likelihood_threshold=likelihood, smooth_window=window_size,
            )
            # Keep what we need; add group label
            df = df[['trial_id', 'head_body_misalignment_p95']].copy().dropna()
            df['group'] = label
            frames.append(df)

        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue

        df = pd.concat(frames, ignore_index=True)

        # Plot
        order = list(group_specs.keys())
        fig, ax = plot_groupwise_bar(
            df,
            y='head_body_misalignment_p95',
            ylabel='head_body_misalignment_p95',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ranksums", "ttest", "anova")  # if supported
        )
        ax.set_title(f"{task_name} | likelihood={likelihood}", pad=20)

        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")


### Single call with fixed hyper parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

smooth_window=None
likelihood_threshold=0.65

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    # "Ghrelin": ghrelin_id,
    "Inhibitory": inh_id,
    "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

frames = []
for label, ids in group_specs.items():
    df = batch_angle_features(
        conn, ids, likelihood_threshold=likelihood_threshold, smooth_window=smooth_window
    )
    # Keep what we need; add group label
    df = df[['trial_id', 'head_body_misalignment_p95']].copy().dropna()
    df['group'] = label
    frames.append(df)

if not frames:
    print("[WARN] No groups provided—skipping this window.")

df = pd.concat(frames, ignore_index=True)

# Plot
order = list(group_specs.keys())
fig, ax = plot_groupwise_bar(
    df,
    y='head_body_misalignment_p95',
    ylabel='head_body_misalignment_p95',
    plot_type='bar',
    show_points=False,
    order=order,
    show_stats=True,
    tests_to_show=("ranksums", "ttest", "anova")  # if supported
)
ax.set_title(f"{task_name} | likelihood_threshold={likelihood_threshold}", pad=20)

fig.savefig(f"White_Modulation_2X_{task_name}_ang.pdf", dpi=300, bbox_inches='tight')
